In [ ]:

# Set up wandb.
import wandb
api = wandb.Api(timeout=60)
wandb.login()

# Set up pandas
import pandas as pd
# set pandas column width
pd.set_option('display.max_colwidth', 500)

from plotting import plot_throughput, plot_completion_rates, plot_weights, plot_dendrite_rates, plot_leaderboard, plot_network_embedding, plot_completion_rewards


# Access the run

In [ ]:
run_id = "kltiefxf" # replace this with your own.
run = api.run(f"/opentensor/opentensor-validator/runs/{run_id}")

In [ ]:
# Reload the history.
df = run.history(samples=100000)
print(f'History for run {run_id!r} contains {len(df)} records. Dataframe shape: {df.shape}')


In [ ]:

df.to_csv(f'history-{run_id}.csv')

In [ ]:
# Convert timestamp to datetime.
df._timestamp = pd.to_datetime(df._timestamp, unit='s')
df.head()

In [ ]:

# List columns
df.info()

In [ ]:
df['best_followup'].value_counts()

## Contents of first log entry

In [ ]:
df.iloc[0]

In [ ]:
# Get number of unique elements and list lengths for ith row
i = 0
pd.DataFrame([{'name':k, 'nunique':len(set(a)), 'length':len(a)} for k, a in df.iloc[i].items() if isinstance(a, list)])

In [ ]:
df.loc[0,'answer_completions']

# Logging throughput

In [ ]:
plot_throughput(df,n_minutes=10)

# Dendrite Response Success Rates

In [ ]:
# plots dendrite completion success rates (for answers by default)
plot_dendrite_rates(df)


In [ ]:
# plot dendrite completion rates for followup (with random uids for illustration purposes)

plot_dendrite_rates(df, uid_col='followup_uids', reward_col='followup_rewards', uids=[11,4,72,105,21,444,51,34,100,200,299])


# Network Weights

In [ ]:
# Create a column for each UID and show most recent rows
scores = df['moving_averaged_scores'].apply(pd.Series)
# use timestamp as index
scores.index = df['_timestamp']
# rename columns
scores.rename({i: f'UID-{i}' for i in range(scores.shape[1])}, axis=1, inplace=True)
# show most recent rows
scores.tail(10)

In [ ]:
# plot_weights(scores, ntop=10)

# plot specific uid weights
plot_weights(scores.iloc[::50], uids=['UID-0','UID-129','UID-313'])

# plot Every 50th row
# plot_weights(scores.iloc[::50]).show()

# Analyze Completion Rates

In [ ]:
# popularity-based completion search
plot_completion_rates(df, msg_col='answer_completions', time_interval='H', ntop=20)

In [ ]:
# specific completion search
completions = [
    'The magnitude of the 2008 Panzhihua earthquake was Ms 6.1.',
    'The reason for Chopin\'s preference for playing at his own Paris apartment for small groups of friends is not mentioned in the given text.'
    ]    
plot_completion_rates(df, msg_col='answer_completions', time_interval='30T', ntop=20, completions=completions)

In [ ]:
# regex-based completion search
plot_completion_rates(df, msg_col='answer_completions', time_interval='H', ntop=20, completion_regex='fuck|baseball')

# Plot Rewards for a Set of Completions

In [ ]:
plot_completion_rewards(df)


In [ ]:

# Show for specific completions
completions = [
    'What other notable achievements has Beyoncé accomplished in her career?',
    'What is the capital city of the Republic of the Congo?',
    'What was the plot of Spectre?'
]
plot_completion_rewards(df, completions=completions)

# Plot Reward Leaderboard

In [ ]:
plot_leaderboard(df)

# Plot Embedding of Completions

In [ ]:

plot_network_embedding(df)